In [1]:
from elasticsearch import Elasticsearch

ELASTIC_PASSWORD = "6_UJ-snFLq_FKHjweJhg"

CERT_FINGERPRINT = "1123dfb3b65fa37733335d06eddf4002585aee390b6e5047fc40e247ba723920"

es = Elasticsearch(
    "https://localhost:9200",
    ssl_assert_fingerprint=CERT_FINGERPRINT,
    basic_auth=("elastic", ELASTIC_PASSWORD)
)

if es.ping():
    print("connected!")

print(es.info()._body)

connected!
{'name': '7cbebeb542a3', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'Dd9uvPVYTAaJV3SHr9HTgA', 'version': {'number': '9.1.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '0c781091a2f57de895a73a1391ff8426c0153c8d', 'build_date': '2025-08-24T22:05:04.526302670Z', 'build_snapshot': False, 'lucene_version': '10.2.2', 'minimum_wire_compatibility_version': '8.19.0', 'minimum_index_compatibility_version': '8.0.0'}, 'tagline': 'You Know, for Search'}


In [2]:
es.indices.delete(index='my_index1', ignore_unavailable=True)
es.indices.create(index='my_index1')

es.indices.delete(index='my_index2', ignore_unavailable=True)
es.indices.create(index='my_index2')

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index2'})

In [4]:
import json

dummy_data = json.load(open('dummy_data.json'))
dummy_data

document_ids = []

def insert_document(my_index, document):
    response = es.index(index=my_index, body=document)
    return response

for document in dummy_data:
    response = insert_document('my_index1', document)
    document_ids.append(response['_id'])

    response = insert_document('my_index2', document)
    document_ids.append(response['_id'])

document_ids

['MEqo9ZgBDVTKBi1fafUJ',
 'MUqo9ZgBDVTKBi1fafUS',
 'Mkqo9ZgBDVTKBi1fafUY',
 'M0qo9ZgBDVTKBi1fafUd',
 'NEqo9ZgBDVTKBi1fafUj',
 'NUqo9ZgBDVTKBi1fafUm']

# Searching

In [7]:
response = es.search(
    index='my_index1',
    body={
        'query': {'match_all': {}}
    }
)

n_hits = response['hits']['total']['value']
print(n_hits)
response

6


ObjectApiResponse({'took': 1, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 6, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'my_index1', '_id': 'Kkqo9ZgBDVTKBi1fYfWc', '_score': 1.0, '_source': {'title': 'title 1', 'text': 'text 1', 'created_on': '2024-08-27'}}, {'_index': 'my_index1', '_id': 'LEqo9ZgBDVTKBi1fYfXM', '_score': 1.0, '_source': {'title': 'title 2', 'text': 'text 2', 'created_on': '2024-08-27'}}, {'_index': 'my_index1', '_id': 'Lkqo9ZgBDVTKBi1fYfXU', '_score': 1.0, '_source': {'title': 'title 3', 'text': 'text 3', 'created_on': '2024-08-27'}}, {'_index': 'my_index1', '_id': 'MEqo9ZgBDVTKBi1fafUJ', '_score': 1.0, '_source': {'title': 'title 1', 'text': 'text 1', 'created_on': '2024-08-27'}}, {'_index': 'my_index1', '_id': 'Mkqo9ZgBDVTKBi1fafUY', '_score': 1.0, '_source': {'title': 'title 2', 'text': 'text 2', 'created_on': '2024-08-27'}}, {'_index': 'my_index1', '_id': 'NEqo9ZgBDVTKBi1fafUj',

In [8]:
response = es.search(
    index='my_index1,my_index2',
    body={
        'query': {'match_all': {}}
    }
)

n_hits = response['hits']['total']['value']
print(n_hits)
response

12


ObjectApiResponse({'took': 7, 'timed_out': False, '_shards': {'total': 2, 'successful': 2, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 12, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'my_index1', '_id': 'Kkqo9ZgBDVTKBi1fYfWc', '_score': 1.0, '_source': {'title': 'title 1', 'text': 'text 1', 'created_on': '2024-08-27'}}, {'_index': 'my_index1', '_id': 'LEqo9ZgBDVTKBi1fYfXM', '_score': 1.0, '_source': {'title': 'title 2', 'text': 'text 2', 'created_on': '2024-08-27'}}, {'_index': 'my_index1', '_id': 'Lkqo9ZgBDVTKBi1fYfXU', '_score': 1.0, '_source': {'title': 'title 3', 'text': 'text 3', 'created_on': '2024-08-27'}}, {'_index': 'my_index1', '_id': 'MEqo9ZgBDVTKBi1fafUJ', '_score': 1.0, '_source': {'title': 'title 1', 'text': 'text 1', 'created_on': '2024-08-27'}}, {'_index': 'my_index1', '_id': 'Mkqo9ZgBDVTKBi1fafUY', '_score': 1.0, '_source': {'title': 'title 2', 'text': 'text 2', 'created_on': '2024-08-27'}}, {'_index': 'my_index1', '_id': 'NEqo9ZgBDVTKBi1fafUj'

In [9]:
response = es.search(
    index='_all',
    body={
        'query': {'match_all': {}}
    }
)

n_hits = response['hits']['total']['value']
print(n_hits)
response

25


ObjectApiResponse({'took': 17, 'timed_out': False, '_shards': {'total': 6, 'successful': 6, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 25, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'binary_index', '_id': 'F0q-8pgBDVTKBi1fmPUV', '_score': 1.0, '_source': {'image_data': '...'}}, {'_index': 'my_documents', '_id': 'hrkN8pgBvTmWfglGRUlR', '_score': 1.0, '_source': {'title': 'title', 'text': 'text', 'created_on': '2025-08-27'}}, {'_index': 'my_documents', '_id': 'Ekqn8pgBDVTKBi1fo_Xl', '_score': 1.0, '_source': {'title': 'title', 'text': 'text', 'created_on': '2025-08-27'}}, {'_index': 'my_documents', '_id': 'GUou85gBDVTKBi1ftfW8', '_score': 1.0, '_source': {'title': 'title', 'text': 'text', 'created_on': '2025-08-27'}}, {'_index': 'my_documents', '_id': 'Gkoy85gBDVTKBi1fPPVF', '_score': 1.0, '_source': {'title': 'title', 'text': 'text', 'created_on': '2025-08-27'}}, {'_index': 'my_documents', '_id': 'G0oy85gBDVTKBi1fafXq', '_score': 1.0, '_source': {'title': 'tit